In [ ]:
import json
import time
import numpy as np
import paho.mqtt.client as mqtt
from stable_baselines3 import DQN

# 📌 Ruta al modelo
MODEL_PATH = "test3_agent"

# 🧠 Cargar modelo
model = DQN.load(MODEL_PATH)
print(f"✅ Agent loaded from {MODEL_PATH}")

# 🧾 Nombres observables y acciones
OBS_NAMES = ["Tin_SW_fmu", "Tin_SE_fmu", "Tin_NW_fmu", "Tin_NE_fmu", "Tout_fmu", "Rad_fmu"]
ACTION_NAMES = ["Shade1_fmu", "Shade2_fmu", "Shade3_fmu", "Shade4_fmu"]
ACTION_BITS = len(ACTION_NAMES)

# 📦 Buffer
obs_buffer = {}

def on_connect(client, userdata, flags, rc):
    print(f"✅ Connected to MQTT broker (rc={rc})")
    client.subscribe("building/observation", qos=1)
    print("✅ Subscribed to topic: building/observation")

def on_obs(client, userdata, msg):
    obs_buffer.update(json.loads(msg.payload))

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_obs
client.connect("mosquitto", 1883)
client.loop_start()

# 🔁 Bucle principal
while True:
    if obs_buffer:
        # 1. Construir array ordenado
        obs_array = np.array([obs_buffer[k] for k in OBS_NAMES], dtype=np.float32).reshape(1, -1)

        # 2. Predecir acción discreta y extraer valor escalar
        discrete_action = model.predict(obs_array, deterministic=True)[0].item()

        # 3. Convertir a binario
        bin_vec = np.array(list(np.binary_repr(discrete_action, width=ACTION_BITS)), dtype=np.float32)
        action_dict = {name: float(bin_vec[i]) for i, name in enumerate(ACTION_NAMES)}

        # 4. Publicar acción
        client.publish("building/action", json.dumps(action_dict), qos=1)
        print(f"[DEBUG] Acción: {discrete_action} → {action_dict}")

        obs_buffer.clear()

    time.sleep(0.001)


✅ Agent loaded from test3_agent
✅ Connected to MQTT broker (rc=0)
✅ Subscribed to topic: building/observation


/tmp/ipykernel_1223/2062600119.py:30: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


[DEBUG] Acción: 12 → {'Shade1_fmu': 1.0, 'Shade2_fmu': 1.0, 'Shade3_fmu': 0.0, 'Shade4_fmu': 0.0}
[DEBUG] Acción: 12 → {'Shade1_fmu': 1.0, 'Shade2_fmu': 1.0, 'Shade3_fmu': 0.0, 'Shade4_fmu': 0.0}
[DEBUG] Acción: 12 → {'Shade1_fmu': 1.0, 'Shade2_fmu': 1.0, 'Shade3_fmu': 0.0, 'Shade4_fmu': 0.0}
[DEBUG] Acción: 12 → {'Shade1_fmu': 1.0, 'Shade2_fmu': 1.0, 'Shade3_fmu': 0.0, 'Shade4_fmu': 0.0}
[DEBUG] Acción: 12 → {'Shade1_fmu': 1.0, 'Shade2_fmu': 1.0, 'Shade3_fmu': 0.0, 'Shade4_fmu': 0.0}
[DEBUG] Acción: 12 → {'Shade1_fmu': 1.0, 'Shade2_fmu': 1.0, 'Shade3_fmu': 0.0, 'Shade4_fmu': 0.0}
[DEBUG] Acción: 12 → {'Shade1_fmu': 1.0, 'Shade2_fmu': 1.0, 'Shade3_fmu': 0.0, 'Shade4_fmu': 0.0}
[DEBUG] Acción: 12 → {'Shade1_fmu': 1.0, 'Shade2_fmu': 1.0, 'Shade3_fmu': 0.0, 'Shade4_fmu': 0.0}
[DEBUG] Acción: 12 → {'Shade1_fmu': 1.0, 'Shade2_fmu': 1.0, 'Shade3_fmu': 0.0, 'Shade4_fmu': 0.0}
[DEBUG] Acción: 12 → {'Shade1_fmu': 1.0, 'Shade2_fmu': 1.0, 'Shade3_fmu': 0.0, 'Shade4_fmu': 0.0}
[DEBUG] Acción: 12 →